# Find relevant laws of chunks in contract

## Get law database

In [ ]:
import json

with open('/content/legal_corpus.json', 'r', encoding='utf-8') as f:
    law_data = json.load(f)
database = []
for laws in law_data:
    law_id = laws['law_id']
    for article in laws['articles']:
        article_id = article['article_id']
        title = article['title']
        text = article['text']
        database.append({"law_id": law_id, "article_id": article_id, "title": title, "text": text})
print(len(database))
print(database[0])

185
{'law_id': '個人情報の保護に関する法律（平成十五年法律第五十七号）/第一章\u3000総則', 'article_id': 1, 'title': '第一条\u3000目的', 'text': 'この法律は、デジタル社会の進展に伴い個人情報の利用が著しく拡大していることに鑑み、個人情報の適正な取扱いに関し、基本理念及び政府による基本方針の作成その他の個人情報の保護に関する施策の基本となる事項を定め、国及び地方公共団体の責務等を明らかにし、個人情報を取り扱う事業者及び行政機関等についてこれらの特性に応じて遵守すべき義務等を定めるとともに、個人情報保護委員会を設置することにより、行政機関等の事務及び事業の適正かつ円滑な運営を図り、並びに個人情報の適正かつ効果的な活用が新たな産業の創出並びに活力ある経済社会及び豊かな国民生活の実現に資するものであることその他の個人情報の有用性に配慮しつつ、個人の権利利益を保護することを目的とする。'}


## Get contract data

In [ ]:
import json

with open('/content/chunked_contract.json', 'r', encoding='utf-8') as f:
    contract_data = json.load(f)

print(len(contract_data))
print(contract_data[0])

341
{'index': 6, 'text': '第１章\u3000総則\n（目的）\n第１条\u3000本個人情報取扱規程（以下「本規程」という。）は、会社が、個人情報の保護に関する法律（平成１５年法律第５７号）（以下「個人情報保護法」といい、個人情報保護法に基づき制定された政令、規則等を含むものとする。）及び行政手続における特定の個人を識別するための番号の利用等に関する法律（平成２５年法律第２７号）（以下「マイナンバー法」といい、マイナンバー法に基づき制定された政令、規則等を含むものとする。）に基づき、会社の取り扱う個人情報の適正な取扱いを確保するために定めるものである。', 'label': 'dieu', 'muc': '第１章\u3000総則', 'cap': '（目的）', 'dieu': '第１条\u3000本個人情報取扱規程（以下「本規程」という。）は、会社が、個人情報の保護に関する法律（平成１５年法律第５７号）（以下「個人情報保護法」といい、個人情報保護法に基づき制定された政令、規則等を含むものとする。）及び行政手続における特定の個人を識別するための番号の利用等に関する法律（平成２５年法律第２７号）（以下「マイナンバー法」といい、マイナンバー法に基づき制定された政令、規則等を含むものとする。）に基づき、会社の取り扱う個人情報の適正な取扱いを確保するために定めるものである。'}


## Config gemini

In [ ]:
# Configure Gemini
import google.generativeai as genai
from google.colab import userdata

system_instruction = '''You are an attorney specializing in labor law. Find all the laws COME FROM the context below that can prove the truth or falsity of the user input statement. Do not use any other laws or information that is not in the context.
Using this JSON schema:
  relevant_law = {"law_id": str, "article_id": int, "title": str, "text": str, "explanation": str, "label": bool}
where "law_id", "article_id", "title", "text" is from context, "explanation" is why you choose that law written in Japanese and "label" is true if the statement is truth proven by that law otherwise is false.
Return a `list[relevant_law]`

If you can't find any laws, just return an empty list

Context: %s
'''

gemini_api_secret_name = 'GOOGLE_API_KEY_1'
GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)

genai.configure(api_key=GOOGLE_API_KEY)
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    }
]
model = genai.GenerativeModel('gemini-1.5-flash-latest',
                              safety_settings=safety_settings,
                              # generation_config={"response_mime_type": "application/json", "max_output_tokens": 8000},
                              system_instruction=system_instruction % (database))
chat = model.start_chat(history=[])

prompt_template = '''
User input: 第３章 個人情報の取扱い 第１節　個人情報の取得・保有等
（利用目的の特定）
第２０条　会社は、業務を遂行するため必要な場合に限り、かつ目的の達成に必要な範囲に限って個人情報を取得するものとし、その利用の目的（以下「利用目的」という。）をできる限り特定するものとする。

Answer: [
    {
        "law_id": "個人情報の保護に関する法律（平成十五年法律第五十七号）/第四章　個人情報取扱事業者等の義務等",
        "article_id": 17,
        "title": "第十七条　利用目的の特定",
        "text": "個人情報取扱事業者は、個人情報を取り扱うに当たっては、その利用の目的（以下「利用目的」という。）をできる限り特定しなければならない。個人情報取扱事業者は、利用目的を変更する場合には、変更前の利用目的と関連性を有すると合理的に認められる範囲を超えて行ってはならない。",
        "explanation": "この法律の第17条は、個人情報取扱事業者は、個人情報を取り扱う際には、その利用目的をできる限り特定しなければならないと定めています。これは、個人関連情報の取扱いにおいても同様であり、個人関連情報を取得、保有、利用する際には、その利用目的を明確に特定する必要があることを示唆しています。",
        "label": true
    }
]

User input: %s

Answer:
'''

list_result = []

## Generating

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash-latest',
                              safety_settings=safety_settings)
chat = model.start_chat(history=[])
response = chat.send_message('hello', stream=True)
response.resolve()
print(response.text)
print('total token:', response.usage_metadata.total_token_count)

Hello! How can I help you today? 

total token: 12


#### one example

In [ ]:
start_index = chunk_index = 2
# chunk_index += 1
print("chunk_index:", chunk_index)
contract = contract_data[chunk_index]
print(contract['text']+'\n')

chunk_index: 2
第１章　総則
（改廃）
第３条　本規程は、従業員の過半数を代表する者の意見を聴取の上、取締役会の決議により、改廃する。



In [ ]:
# contract = {}

In [ ]:
# start_index = chunk_index = 51
# chunk_index += 1
# contract = contract_data[chunk_index]
response = chat.send_message(prompt_template % (contract['text']), stream=True)
# response = model.generate_content(prompt_template % (contract['text']), stream=True)
# response = model.generate_content(system_instruction % (database[:len(database)//2]) + prompt_template % (contract['text']), stream=True)
# gemini_response  = ''.join([chunk.text for chunk in response]).strip().replace('```json', '').replace('```', '').replace("\n", '').replace(" ", '')
response.resolve()
gemini_response = response.text.strip().replace('```json', '').replace('```', '')
gemini_response = json.loads(gemini_response)
list_result.append({"question": contract, "relevant_laws": gemini_response})
print('total token:', response.usage_metadata.total_token_count)
# print("chunk_index:", chunk_index)
print(contract['text']+'\n')
gemini_response

In [ ]:
# print("chunk_index:", chunk_index)
# print(contract['text']+'\n')
# print(response.text.strip().replace('```json', '').replace('```', ''))

In [ ]:
with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(list_result[-1], f, ensure_ascii=False, indent=4)

In [ ]:
last_sent, last_received = chat.rewind()
print(len(chat.history))

copy thủ công phần response gán cho `relevevant_laws` dưới dạng `str`

In [ ]:
relevant_laws = ''''''
list_result.append({"question": contract, "relevant_laws": json.loads(relevant_laws.replace("\n", ''))})

#### full example

In [ ]:
import time

start_index = 9
chunk_index = 0
for contract in contract_data[start_index:]:
    print("chunk_index:", start_index+chunk_index, '/', len(contract_data) - 1)
    print(contract['text']+'\n')
    # response = chat.send_message(prompt_template % (contract['text']), stream=True)
    response = model.generate_content(prompt_template % (contract['text']), stream=True)
    gemini_response  = ''.join([chunk.text for chunk in response]).strip().replace('```json', '').replace('```', '').replace("\n", '').replace(" ", '')
    gemini_response = json.loads(gemini_response)
    list_result.append({"question": contract, "relevant_laws": gemini_response})
    chunk_index += 1
    if chunk_index % 15 == 0:
        time.sleep(60)

chunk_index: 9 / 43
第２章　入舎及び退舎
（入舎及び退舎の手続等）
第８条　入舎する者は、管理者の許可を受けなければならない。

chunk_index: 10 / 43
第２章　入舎及び退舎
（入舎及び退舎の手続等）
２　前項により入舎を許可された者は管理者に所要の事項を届出なければならない。

chunk_index: 11 / 43
第２章　入舎及び退舎
（入舎及び退舎の手続等）
３　業務の都合により、他の寄宿舎に転居を命ぜられた者は、特別な事情がない限り速やかにその命令に従わなければならない。

chunk_index: 12 / 43
第２章　入舎及び退舎
（入舎及び退舎の手続等）
４　退舎しようとする者は事前に管理者に届出なければならない。

chunk_index: 13 / 43
第２章　入舎及び退舎
（入舎及び退舎の手続等）
５　退職者は、速やかに退舎しなければならない。また、法令等により、寄宿不適当と判断された者には、退舎を命ずることがある。

chunk_index: 14 / 43
第２章　入舎及び退舎
（入舎及び退舎の手続等）
６　前２項により退舎する寄宿員は、借受けた物品及び入居していた室について、速やかに返却するとともに、管理者の点検を受けなければならない。

chunk_index: 15 / 43
第３章　日課
（起床・就寝時刻）
第９条　起床時刻は午前○時○分とし、就寝時刻は午後○時○分とする。ただし、業務の都合により、この時刻を変更することがある。

chunk_index: 16 / 43
第３章　日課
（起床・就寝時刻）
２　前項ただし書により、時刻の変更を受けた者は、他の寄宿員の迷惑にならないよう心がけなければならない。

chunk_index: 17 / 43
第３章　日課
（食事時間等）
第１０条　食事時刻は、次のとおりとする。
朝食　午前○時○分から午前○時○分まで
昼食　午後○時○分から午後○時○分まで
夕食　午後○時○分から午後○時○分まで

chunk_index: 18 / 43
第３章　日課
（食事時間等）
２　食事は、食堂以外の場所でしてはならない。

chunk_index: 19 / 43
第３章　日課
（食事時間等）
３　管理者は、常に食生活に留意し、炊事場、食器類等を清潔に保

In [ ]:
import time

start_index = 0
chunk_index = 0
for contract in contract_data[start_index:8]:
    print("chunk_index:", start_index+chunk_index, '/', len(contract_data) - 1)
    print(contract['text']+'\n')
    # response = chat.send_message(prompt_template % (contract['text']), stream=True)
    response = model.generate_content(prompt_template % (contract['text']), stream=True)
    gemini_response  = ''.join([chunk.text for chunk in response]).strip().replace('```json', '').replace('```', '').replace("\n", '').replace(" ", '')
    gemini_response = json.loads(gemini_response)
    list_result.append({"question": contract, "relevant_laws": gemini_response})
    chunk_index += 1
    if chunk_index % 15 == 0:
        time.sleep(60)

chunk_index: 0 / 43
第１章　総則
（目的）
第１条　本寄宿舎規則（以下「規則」という。）は、労働基準法（以下「労基法」という。）第９５条第１項により、寄宿舎生活の秩序について定めるものであり、当該寄宿舎における集団生活としての規律を守り、健康で文化的な私生活を営むことを目的とする。

chunk_index: 1 / 43
第１章　総則
（適用範囲）
第２条　本規則は株式会社○○○（以下「会社」という。）に付属する寄宿舎に適用する。

chunk_index: 2 / 43
第１章　総則
（規則の遵守）
第３条　会社と寄宿舎に寄宿する労働者（以下「寄宿員」という。）は、ともに本規則を遵守するほか、安全及び衛生に関する法令及び会社の定めた諸規則を守らなければならない。

chunk_index: 3 / 43
第１章　総則
（規則責任者）
第４条　本規則の管理責任者は、総務部長とする。

chunk_index: 4 / 43
第１章　総則
（解釈上の疑義）
第５条　本規則及びこれに付属する諸規程等の解釈について疑義が生じた場合は、総務部長は関係部署の長と協議の上、これを決定する。

chunk_index: 5 / 43
第１章　総則
（改廃）
第６条　本規則は、寄宿員の過半数を代表する者の同意を得て、改廃する。

chunk_index: 6 / 43
第１章　総則
（改廃）
２　会社は本規則を変更しようとするときは、その案をあらかじめ寄宿員に周知させる措置を講ずるものとする。

chunk_index: 7 / 43
第１章　総則
（管理者）
第７条　寄宿員の事業主は、会社であり、寄宿舎及びその付属施設並びに寄宿員に関する事項は、会社、主任〇〇〇（以下「管理者」という。）が管理する。



In [ ]:
# if stream broke, delete the last conversation
last_sent, last_received = chat.rewind()

In [ ]:
print(response.text)

In [ ]:
print(len(list_result))

44


Nếu vẫn lỗi => save data rồi restart runtime, thay api key, sau đó chạy tiếp từ index gây lỗi

## Save data

In [ ]:
import json

with open('/content/find_law_gemini_contract.json', 'w', encoding='utf-8') as f:
    json.dump(list_result, f, ensure_ascii=False, indent=4)

In [ ]:
import json

with open('/content/find_law_gemini_contract.json', 'r') as f:
    list_result = json.load(f)
print(len(list_result))
print(list_result[0])

44
{'question': {'index': 3, 'text': '第１章\u3000総則\n（目的）\n第１条\u3000本寄宿舎規則（以下「規則」という。）は、労働基準法（以下「労基法」という。）第９５条第１項により、寄宿舎生活の秩序について定めるものであり、当該寄宿舎における集団生活としての規律を守り、健康で文化的な私生活を営むことを目的とする。', 'label': 'dieu', 'muc': '第１章\u3000総則', 'cap': '（目的）', 'dieu': '第１条\u3000本寄宿舎規則（以下「規則」という。）は、労働基準法（以下「労基法」という。）第９５条第１項により、寄宿舎生活の秩序について定めるものであり、当該寄宿舎における集団生活としての規律を守り、健康で文化的な私生活を営むことを目的とする。'}, 'relevant_laws': [{'law_id': '労働基準法/第十一章\u3000寄宿舎', 'article_id': 95, 'title': '第九十五条\u3000寄宿舎規則', 'text': '使用者は、寄宿舎に労働者を寄宿させる場合においては、寄宿舎生活の秩序を保持するため、寄宿舎規則を定めなければならない。\n２\u3000寄宿舎規則には、次に掲げる事項を記載しなければならない。\n一\u3000寄宿舎生活の秩序に関する事項\n二\u3000寄宿舎の管理に関する事項\n三\u3000寄宿労働者の義務に関する事項\n四\u3000寄宿舎の管理について権限を有する者の氏名\n五\u3000寄宿労働者の私生活の自由を侵す行為をしてはならない旨\n３\u3000使用者は、寄宿舎規則を作成し、又は変更しようとするときは、その案を寄宿労働者の過半数を代表する者の同意を得なければならない。\n４\u3000使用者は、寄宿労働者に寄宿舎規則を示さなければならない。', 'explanation': '労働基準法第95条は、寄宿舎規則の内容について定めているため、寄宿舎規則の内容が労働基準法第95条で定められた項目に準拠しているかどうかを確認する必要があります。', 'label': True}]}
